In [24]:
import pandas as pd
import ccxt
import os
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
load_dotenv()

APCA_API_KEY_ID = os.getenv('APCA_API_KEY_ID')
APCA_API_SECRET_KEY = os.getenv('APCA_API_SECRET_KEY')


In [12]:

kraken = ccxt.kraken({
    'apiKey': os.getenv('KRAKEN_API_KEY_ID'),
    'secret': os.getenv('KRAKEN_API_SECRET_KEY'),
})


markets = kraken.load_markets()

#print(markets)
import random

print(kraken)
if (kraken.has['fetchTicker']):
    print(kraken.fetch_ticker('ETH/USD')) 
    symbols = list(kraken.markets.keys())
    print(kraken.fetch_ticker(random.choice(symbols))) 

Kraken
{'symbol': 'ETH/USD', 'timestamp': 1648920285967, 'datetime': '2022-04-02T17:24:45.967Z', 'high': 3531.99, 'low': 3428.08, 'bid': 3483.28, 'bidVolume': None, 'ask': 3483.3, 'askVolume': None, 'vwap': 3477.44458, 'open': 3454.65, 'close': 3483.27, 'last': 3483.27, 'previousClose': None, 'change': 28.62, 'percentage': 0.8284486127393513, 'average': 3468.96, 'baseVolume': 33909.33586682, 'quoteVolume': 117917836.22147281, 'info': {'a': ['3483.30000', '1', '1.000'], 'b': ['3483.28000', '5', '5.000'], 'c': ['3483.27000', '0.00265000'], 'v': ['23542.20824315', '33909.33586682'], 'p': ['3488.86466', '3477.44458'], 't': ['25175', '31456'], 'l': ['3440.46000', '3428.08000'], 'h': ['3531.99000', '3531.99000'], 'o': '3454.65000'}}
{'symbol': 'YGG/USD', 'timestamp': 1648920288976, 'datetime': '2022-04-02T17:24:48.976Z', 'high': 3.552, 'low': 3.165, 'bid': 3.215, 'bidVolume': None, 'ask': 3.222, 'askVolume': None, 'vwap': 3.33788, 'open': 3.236, 'close': 3.237, 'last': 3.237, 'previousClose'

In [27]:

api = REST()

#Facebook, Amazon, Apple, Netflix, Google, Microsoft, Uber, Lyft, Airbnb

FANNGMULA_tickers = ["META", "AMZN", "AAPL", "NTFX", "GOOG", "MSFT", "UBER", "LYFT", "ABNB"]

api.get_bars(FANNGMULA_tickers, TimeFrame(45, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-06-08 07:30:00+00:00,126.10,126.16,125.9600,126.06,20951,304,126.049447,AAPL
2021-06-08 08:15:00+00:00,126.05,126.30,126.0500,126.30,21181,349,126.231904,AAPL
2021-06-08 09:00:00+00:00,126.27,126.32,126.2200,126.28,15955,308,126.284120,AAPL
2021-06-08 09:45:00+00:00,126.29,126.40,125.9000,125.90,30179,582,126.196877,AAPL
2021-06-08 10:30:00+00:00,125.90,126.75,125.8400,126.75,105380,1376,126.530863,AAPL
...,...,...,...,...,...,...,...,...
2021-06-08 20:15:00+00:00,49.81,49.88,49.7800,49.86,21235,97,49.860162,UBER
2021-06-08 21:00:00+00:00,49.86,49.88,49.8600,49.86,44362,30,49.879444,UBER
2021-06-08 21:45:00+00:00,49.86,49.88,49.8000,49.85,1739,12,49.864410,UBER
